In [4]:
import dgl
import dgl.function as fn
import torch
import torch.nn as nn

import sys
sys.path.append('/Users/vratinsrivastava/Desktop/research/graphpocket/gnn')

from gnn import dataloader, graphpocket


ModuleNotFoundError: No module named 'gnn'